Change point
1. Combine XGB and Logistec
<br>
<h2>Accuracy : 0.77751</h2>

In [8]:
import numpy as np
import pandas as pd

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/gender_submission.csv')

In [9]:
train_x = train.drop(['Survived'],axis=1)
train_y = train['Survived']
maindata = pd.concat([train_x,test],sort=False)

In [10]:
maindata.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
maindata.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)
maindata['Embarked'].fillna('S',inplace=True)

In [12]:
maindata['Embarked'].value_counts()

S    916
C    270
Q    123
Name: Embarked, dtype: int64

In [13]:
maindata.replace(['male','female'],[0,1],inplace=True)
maindata.replace(['S','C','Q'],[0,1,2],inplace=True)

In [14]:
train_x = maindata[:len(train)]
test_x = maindata[len(train):]

In [15]:
import itertools
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
test_x.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.5,0,0,7.8292,2
1,893,3,1,47.0,1,0,7.0000,0
2,894,2,0,62.0,0,0,9.6875,2
3,895,3,0,27.0,0,0,8.6625,0
4,896,3,1,22.0,1,1,12.2875,0


In [17]:
param_space = {
    'max_depth':[3,5,7],
    'min_child_weight':[1.0,2.0,4.0]
}

In [18]:
#Combine each params and save score to it.
param_combinations = itertools.product(param_space['max_depth'],param_space['min_child_weight'])
params = []
scores = []

In [19]:
# 元データをコピーする
train_x2 = train.drop(['Survived'], axis=1)
test_x2 = test.copy()

# 変数PassengerIdを除外する
train_x2 = train_x2.drop(['PassengerId'], axis=1)
test_x2 = test_x2.drop(['PassengerId'], axis=1)

# 変数Name, Ticket, Cabinを除外する
train_x2 = train_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x2 = test_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# one-hot encodingを行う
cat_cols = ['Sex', 'Embarked', 'Pclass']
ohe = OneHotEncoder(categories='auto', sparse=False)
ohe.fit(train_x2[cat_cols].fillna('NA'))

# one-hot encodingのダミー変数の列名を作成する
ohe_columns = []
for i, c in enumerate(cat_cols):
    ohe_columns += [f'{c}_{v}' for v in ohe.categories_[i]]

# one-hot encodingによる変換を行う
ohe_train_x2 = pd.DataFrame(ohe.transform(train_x2[cat_cols].fillna('NA')), columns=ohe_columns)
ohe_test_x2 = pd.DataFrame(ohe.transform(test_x2[cat_cols].fillna('NA')), columns=ohe_columns)

# one-hot encoding済みの変数を除外する
train_x2 = train_x2.drop(cat_cols, axis=1)
test_x2 = test_x2.drop(cat_cols, axis=1)

# one-hot encodingで変換された変数を結合する
train_x2 = pd.concat([train_x2, ohe_train_x2], axis=1)
test_x2 = pd.concat([test_x2, ohe_test_x2], axis=1)

# 数値変数の欠損値を学習データの平均で埋める
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']
for col in num_cols:
    train_x2[col].fillna(train_x2[col].mean(), inplace=True)
    test_x2[col].fillna(train_x2[col].mean(), inplace=True)

# 変数Fareを対数変換する
train_x2['Fare'] = np.log1p(train_x2['Fare'])
test_x2['Fare'] = np.log1p(test_x2['Fare'])

In [20]:
train_x2.head()

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,2.110213,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,38.0,1,0,4.280593,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,26.0,0,0,2.188856,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,35.0,1,0,3.990834,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,35.0,0,0,2.202765,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [21]:
from sklearn.linear_model import LogisticRegression

# xgboostモデル
model_xgb = XGBClassifier(n_estimators=20, random_state=71)
model_xgb.fit(train_x, train_y)
pred_xgb = model_xgb.predict_proba(test_x)[:, 1]

# ロジスティック回帰モデル
# xgboostモデルとは異なる特徴量を入れる必要があるので、別途train_x2, test_x2を作成した
model_lr = LogisticRegression(solver='lbfgs', max_iter=300)
model_lr.fit(train_x2, train_y)
pred_lr = model_lr.predict_proba(test_x2)[:, 1]

# 予測値の加重平均をとる
pred = pred_xgb * 0.8 + pred_lr * 0.2
pred_label = np.where(pred > 0.5, 1, 0)

In [23]:
pred_label[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0])

In [26]:
submission = pd.DataFrame({'PassengerId':test_x['PassengerId'],'Survived':pred_label})
submission.to_csv('submission20201106.csv',index=False)